In [13]:
%pip install numpy==1.23.3


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-p


     ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
      --------------------------------------- 0.3/10.7 MB ? eta -:--:--
     ----- ---------------------------------- 1.6/10.7 MB 7.0 MB/s eta 0:00:02
     ----------------- ---------------------- 4.7/10.7 MB 9.8 MB/s eta 0:00:01
     --------------------------- ------------ 7.3/10.7 MB 10.3 MB/s eta 0:00:01
     ------------------------------------ --- 9.7/10.7 MB 10.6 MB/s eta 0:00:01
     --------------------------------------- 10.7/10.7 MB 10.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [4]:
class KohonenLayer(nn.Module):
    def __init__(self, input_dim, num_clusters):
        super(KohonenLayer, self).__init__()
        self.weights = nn.Parameter(torch.randn(num_clusters, input_dim))

    def forward(self, x):
        distances = torch.cdist(x.unsqueeze(0), self.weights.unsqueeze(0)).squeeze(0)    
        return distances #torch.argmin(distances, dim=1)

class CounterPropagationNetwork(nn.Module):
    def __init__(self, input_dim, num_clusters, classification_dim, output_dim):
        super(CounterPropagationNetwork, self).__init__()
        self.kohonen = KohonenLayer(input_dim, num_clusters)
        self.associative_layer = nn.Linear(num_clusters, classification_dim)
        self.classification_layer = nn.Linear(classification_dim, output_dim)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.kohonen(x)
        x = self.activation(self.associative_layer(x))
        x = self.classification_layer(x)
        return x


In [5]:
X, y = make_classification(n_samples=1600, n_features=6, n_informative=4, n_classes=3, random_state=56)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=56)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)


In [6]:
input_dim = X_train.shape[1]
num_clusters = 12
output_dim = len(torch.unique(y_train))
classification_dim = num_clusters*2
epochs = 3000

model = CounterPropagationNetwork(input_dim, num_clusters, classification_dim, output_dim).to('cpu')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.9)


train_accuracy = []
val_accuracy = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    train_predictions = torch.argmax(outputs, dim=1)
    accuracy = accuracy_score(y_train.cpu().numpy(), train_predictions.cpu().numpy())
    
    scheduler.step()
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test)
        val_predictions = torch.argmax(val_outputs, dim=1)
        v_accuracy = accuracy_score(y_test.cpu().numpy(), val_predictions.cpu().numpy())
        
    if (epoch + 1) % 100 == 0:
        train_accuracy.append(train_accuracy)
        val_accuracy.append(val_accuracy)
        print(f"Epoch {epoch + 1}\nLoss: {loss.item():.4f}, Train Accuracy: {accuracy:.4f}, Test Accuracy: {v_accuracy:.4f}")


Epoch 100
Loss: 0.5836, Train Accuracy: 0.7433, Test Accuracy: 0.7350
Epoch 200
Loss: 0.4422, Train Accuracy: 0.8317, Test Accuracy: 0.8375
Epoch 300
Loss: 0.3709, Train Accuracy: 0.8742, Test Accuracy: 0.8450
Epoch 400
Loss: 0.2758, Train Accuracy: 0.9100, Test Accuracy: 0.8750
Epoch 500
Loss: 0.2250, Train Accuracy: 0.9250, Test Accuracy: 0.8850
Epoch 600
Loss: 0.1986, Train Accuracy: 0.9367, Test Accuracy: 0.8950
Epoch 700
Loss: 0.1809, Train Accuracy: 0.9450, Test Accuracy: 0.9050
Epoch 800
Loss: 0.1664, Train Accuracy: 0.9558, Test Accuracy: 0.9050
Epoch 900
Loss: 0.1556, Train Accuracy: 0.9567, Test Accuracy: 0.9125
Epoch 1000
Loss: 0.1471, Train Accuracy: 0.9583, Test Accuracy: 0.9175
Epoch 1100
Loss: 0.1392, Train Accuracy: 0.9600, Test Accuracy: 0.9175
Epoch 1200
Loss: 0.1326, Train Accuracy: 0.9625, Test Accuracy: 0.9250
Epoch 1300
Loss: 0.1272, Train Accuracy: 0.9633, Test Accuracy: 0.9225
Epoch 1400
Loss: 0.1226, Train Accuracy: 0.9642, Test Accuracy: 0.9250
Epoch 1500
Loss

In [ ]:
plt.figure(figsize=(8, 5))
plt.title("Valid accuracy")
plt.plot(range(epochs), val_accuracy, label="CPN test loss", linewidth=2)
plt.legend()
plt.xlabel("Epoch")
plt.show()